# 🎙️ Chatterbox TTS - Google Colab Edition

High-quality voice cloning, text-to-speech & voice conversion powered by Chatterbox.

**Features:**
- ⚡ **Turbo TTS** - Fast, high-quality speech synthesis with paralinguistic tags
- 🎤 **TTS Main** - Advanced English TTS with fine-tuned controls
- 🌍 **Multilingual TTS** - Support for multiple languages
- 🔄 **Voice Conversion** - Convert any voice to your target voice
- 🧬 **Voice Cloning** - Clone voices from audio samples

---

## 📦 Step 1: Setup Environment

This will install all required dependencies. Takes ~3-5 minutes.

In [ ]:
%%capture
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq ffmpeg

# Install Python packages
!pip install -q numpy>=1.24.0,<1.26.0
!pip install -q librosa==0.11.0
!pip install -q s3tokenizer
!pip install -q torch==2.7.1 torchaudio==2.7.1 --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers==4.46.3
!pip install -q diffusers==0.29.0
!pip install -q resemble-perth==1.0.1
!pip install -q conformer==0.3.2
!pip install -q safetensors==0.5.3
!pip install -q pykakasi==2.3.0
!pip install -q gradio==5.44.1
!pip install -q pyloudnorm
!pip install -q omegaconf
!pip install -q huggingface_hub

print("✅ All dependencies installed successfully!")

## 📥 Step 2: Clone Repository

Download the Chatterbox TTS application code.

In [ ]:
import os

# Clone the repository (replace with your actual repo URL)
if not os.path.exists('chatterbox-tts'):
    !git clone https://github.com/YOUR_USERNAME/chatterbox-tts.git
    print("✅ Repository cloned!")
else:
    print("ℹ️ Repository already exists")

# Change to project directory
%cd chatterbox-tts

### Alternative: Upload Files Manually

If you don't have a GitHub repo, run this cell to upload your project files:

In [ ]:
# Uncomment and run this if you want to upload files manually
# from google.colab import files
# import zipfile

# print("Please upload your project as a ZIP file:")
# uploaded = files.upload()

# for filename in uploaded.keys():
#     print(f"Extracting {filename}...")
#     with zipfile.ZipFile(filename, 'r') as zip_ref:
#         zip_ref.extractall('.')
#     print("✅ Files extracted!")

## 🔍 Step 3: Verify Installation

Check if everything is set up correctly.

In [ ]:
import sys
import torch

# Check CUDA availability
print(f"🔧 Python version: {sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"💾 CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Verify project structure
required_files = ['app.py', 'requirements.txt', 'src', 'modules']
for item in required_files:
    if os.path.exists(item):
        print(f"✅ Found: {item}")
    else:
        print(f"❌ Missing: {item}")

print("\n✅ Setup verification complete!")

## 🤖 Step 4: Download Models (Optional Pre-download)

Pre-download the models to speed up first use. This is optional - models will auto-download when first used.

In [ ]:
from huggingface_hub import snapshot_download

print("📥 Downloading Chatterbox Turbo model...")
snapshot_download(
    repo_id="ResembleAI/chatterbox-turbo",
    token=False,  # Use public access without authentication
    allow_patterns=["*.safetensors", "*.json", "*.txt", "*.pt", "*.model"]
)
print("✅ Turbo model downloaded!")

print("\n📥 Downloading Chatterbox main model...")
from huggingface_hub import hf_hub_download
for fpath in ["ve.safetensors", "t3_cfg.safetensors", "s3gen.safetensors", "tokenizer.json", "conds.pt"]:
    hf_hub_download(repo_id="ResembleAI/chatterbox", filename=fpath)
print("✅ Main model downloaded!")

print("\n🎉 All models ready!")

## 🚀 Step 5: Launch Application

Start the Gradio interface. Click the public URL to access the app!

In [ ]:
# Add project paths
import sys
import os

project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.append(project_root)

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Import and launch
import gradio as gr
from modules.config import LANGUAGE_CONFIG, SUPPORTED_LANGUAGES
from modules.voice_manager import (
    load_voices, 
    get_voices_for_language, 
    get_all_voices_with_gender,
    resolve_voice_path,
    clone_voice,
    delete_voice
)
from modules.generation_functions import (
    generate_speech,
    generate_multilingual_speech,
    convert_voice,
    generate_turbo_speech
)
from modules.ui_components import (
    create_header,
    create_tts_tab,
    create_multilingual_tab,
    create_voice_conversion_tab,
    create_clone_voice_tab,
    create_turbo_tab
)

# Load voices
available_voices = load_voices()

# Custom CSS
CUSTOM_CSS = """
.tag-container {
    display: flex !important;
    flex-wrap: wrap !important;
    gap: 8px !important;
    margin-top: 5px !important;
    margin-bottom: 10px !important;
    border: none !important;
    background: transparent !important;
}
.tag-btn {
    min-width: fit-content !important;
    width: auto !important;
    height: 32px !important;
    font-size: 13px !important;
    background: #eef2ff !important;
    border: 1px solid #c7d2fe !important;
    color: #3730a3 !important;
    border-radius: 6px !important;
    padding: 0 10px !important;
    margin: 0 !important;
    box-shadow: none !important;
}
.tag-btn:hover {
    background: #c7d2fe !important;
    transform: translateY(-1px);
}
"""

# Create Gradio app
with gr.Blocks(title="Chatterbox TTS Enhanced", theme=gr.themes.Soft(), css=CUSTOM_CSS) as demo:
    # State variables
    tts_model_state = gr.State(None)
    vc_model_state = gr.State(None)
    mtl_model_state = gr.State(None)
    
    # Header
    create_header()
    
    # Create tabs
    with gr.Tab("⚡ Turbo TTS"):
        turbo_components = create_turbo_tab()

    with gr.Tab("🎤 TTS Main (English)"):
        tts_components = create_tts_tab()
    
    with gr.Tab("🌍 Multilingual TTS"):
        mtl_components = create_multilingual_tab()
    
    with gr.Tab("🔄 Voice Conversion"):
        vc_components = create_voice_conversion_tab()
    
    with gr.Tab("🧬 Clone Voice"):
        clone_components = create_clone_voice_tab()
    
    # Event Handlers - TTS Tab
    tts_components['generate_btn'].click(
        fn=generate_speech,
        inputs=[
            tts_components['text'],
            tts_components['voice_select'],
            tts_components['exaggeration'],
            tts_components['temp'],
            tts_components['seed_num'],
            tts_components['cfg_weight'],
            tts_components['min_p'],
            tts_components['top_p'],
            tts_components['repetition_penalty']
        ],
        outputs=[
            tts_components['progress_bar'],
            tts_components['audio_output'],
            tts_components['status_box']
        ]
    )
    
    def update_tts_preview(voice_name):
        path = resolve_voice_path(voice_name, "en")
        return path

    tts_components['voice_select'].change(
        fn=update_tts_preview,
        inputs=[tts_components['voice_select']],
        outputs=[tts_components['preview_audio']]
    )
    
    # Event Handlers - Turbo Tab
    turbo_components['generate_btn'].click(
        fn=generate_turbo_speech,
        inputs=[
            turbo_components['text'],
            turbo_components['voice_select']
        ],
        outputs=[
            turbo_components['progress_bar'],
            turbo_components['audio_output'],
            turbo_components['status_box']
        ]
    )
    
    def update_turbo_preview(voice_name):
        path = resolve_voice_path(voice_name, "en")
        return path

    turbo_components['voice_select'].change(
        fn=update_turbo_preview,
        inputs=[turbo_components['voice_select']],
        outputs=[turbo_components['preview_audio']]
    )
    
    # Tag insertion buttons (Turbo)
    INSERT_TAG_JS = """
    (tag_val, current_text) => {
        const textarea = document.querySelector('#turbo_textbox textarea');
        if (!textarea) return current_text + " " + tag_val;
        const start = textarea.selectionStart;
        const end = textarea.selectionEnd;
        let prefix = " ";
        let suffix = " ";
        if (start === 0) prefix = "";
        else if (current_text[start - 1] === ' ') prefix = "";
        if (end < current_text.length && current_text[end] === ' ') suffix = "";
        return current_text.slice(0, start) + prefix + tag_val + suffix + current_text.slice(end);
    }
    """
    
    # Tag button handlers
    for tag_name in ['clear_throat', 'sigh', 'shush', 'cough', 'groan', 'sniff', 'gasp', 'chuckle', 'laugh']:
        btn_key = f'btn_{tag_name}'
        if btn_key in turbo_components:
            turbo_components[btn_key].click(
                fn=None,
                inputs=[turbo_components[btn_key], turbo_components['text']],
                outputs=turbo_components['text'],
                js=INSERT_TAG_JS
            )
    
    # Event Handlers - Multilingual Tab
    mtl_components['generate_btn'].click(
        fn=generate_multilingual_speech,
        inputs=[
            mtl_components['text'],
            mtl_components['voice_select'],
            mtl_components['language_select'],
            mtl_components['exaggeration'],
            mtl_components['temp'],
            mtl_components['seed_num'],
            mtl_components['cfg_weight']
        ],
        outputs=[
            mtl_components['progress_bar'],
            mtl_components['audio_output'],
            mtl_components['status_box']
        ]
    )
    
    mtl_components['language_select'].change(
        fn=lambda lang: (
            LANGUAGE_CONFIG.get(lang, {}).get("text", ""),
            gr.update(choices=get_voices_for_language(lang), value=f"Default ({SUPPORTED_LANGUAGES.get(lang, lang)})")
        ),
        inputs=[mtl_components['language_select']],
        outputs=[mtl_components['text'], mtl_components['voice_select']]
    )
    
    def update_mtl_preview(voice_name, language_code):
        path = resolve_voice_path(voice_name, language_code)
        return path

    mtl_components['voice_select'].change(
        fn=update_mtl_preview,
        inputs=[mtl_components['voice_select'], mtl_components['language_select']],
        outputs=[mtl_components['sample_audio']]
    )
    
    # Event Handlers - Voice Conversion Tab
    vc_components['convert_btn'].click(
        fn=convert_voice,
        inputs=[vc_components['input_audio'], vc_components['target_voice_select']],
        outputs=[vc_components['progress_bar'], vc_components['audio_output'], vc_components['status_box']]
    )
    
    def update_vc_preview(voice_name):
        if voice_name == "None": 
            return None
        
        clean_name = voice_name.replace(" ♂️", "").replace(" ♀️", "")
        
        if clean_name.startswith("Default ("):
            lang_name = clean_name.split("(")[1].split(")")[0]
            for code, name in SUPPORTED_LANGUAGES.items():
                if name == lang_name:
                    return LANGUAGE_CONFIG.get(code, {}).get("audio")
        
        from modules.voice_manager import VOICES
        possible_names = [
            clean_name,
            f"{clean_name}_male",
            f"{clean_name}_female"
        ]
        
        for name in possible_names:
            if name in VOICES["samples"]:
                return VOICES["samples"][name]
        
        for code in SUPPORTED_LANGUAGES:
            for name in possible_names:
                full_name = f"{name}_{code}"
                if full_name in VOICES["samples"]:
                    return VOICES["samples"][full_name]
        
        return None

    vc_components['target_voice_select'].change(
        fn=update_vc_preview,
        inputs=[vc_components['target_voice_select']],
        outputs=[vc_components['preview_audio']]
    )
    
    # Event Handlers - Clone Voice Tab
    clone_components['clone_btn'].click(
        fn=clone_voice,
        inputs=[
            clone_components['ref_audio_input'],
            clone_components['new_voice_name'],
            clone_components['voice_language'],
            clone_components['voice_gender']
        ],
        outputs=[clone_components['clone_status'], tts_components['voice_select']]
    ).then(
        fn=lambda: gr.update(choices=get_voices_for_language("en")),
        outputs=[tts_components['voice_select']]
    ).then(
        fn=lambda: gr.update(choices=get_voices_for_language("en")),
        outputs=[turbo_components['voice_select']]
    ).then(
        fn=lambda lang: gr.update(choices=get_voices_for_language(lang)),
        inputs=[mtl_components['language_select']],
        outputs=[mtl_components['voice_select']]
    ).then(
        fn=lambda: gr.update(choices=["None"] + get_all_voices_with_gender()),
        outputs=[vc_components['target_voice_select']]
    ).then(
        fn=lambda: "\n".join(load_voices()) if load_voices() else "No voices cloned yet",
        outputs=[clone_components['current_voices_display']]
    ).then(
        fn=lambda: gr.update(choices=["None"] + get_all_voices_with_gender(), value="None"),
        outputs=[clone_components['voice_to_delete']]
    )
    
    clone_components['delete_btn'].click(
        fn=delete_voice,
        inputs=[clone_components['voice_to_delete']],
        outputs=[clone_components['delete_status'], clone_components['voice_to_delete']]
    ).then(
        fn=lambda: gr.update(choices=get_voices_for_language("en")),
        outputs=[tts_components['voice_select']]
    ).then(
        fn=lambda: gr.update(choices=get_voices_for_language("en")),
        outputs=[turbo_components['voice_select']]
    ).then(
        fn=lambda lang: gr.update(choices=get_voices_for_language(lang)),
        inputs=[mtl_components['language_select']],
        outputs=[mtl_components['voice_select']]
    ).then(
        fn=lambda: gr.update(choices=["None"] + get_all_voices_with_gender()),
        outputs=[vc_components['target_voice_select']]
    ).then(
        fn=lambda: "\n".join(load_voices()) if load_voices() else "No voices cloned yet",
        outputs=[clone_components['current_voices_display']]
    )

# Launch with public sharing enabled
print("\n🚀 Launching Chatterbox TTS...\n")
demo.queue(
    max_size=50,
    default_concurrency_limit=1,
).launch(
    share=True,  # Create public URL
    debug=True,
    show_error=True
)

## 💡 Tips & Troubleshooting

### Common Issues:

1. **Out of Memory Error**
   - Use a smaller batch size
   - Restart runtime and clear outputs
   - Use Colab Pro for more RAM/GPU

2. **Model Download Fails**
   - Check internet connection
   - Verify HuggingFace is accessible
   - Try running Step 4 again

3. **Import Errors**
   - Restart runtime
   - Re-run Step 1 (dependencies)
   - Check if all files are uploaded

### Performance Tips:

- **Use GPU**: Runtime → Change runtime type → GPU (T4)
- **Keep session alive**: Install Colab Keep Alive extension
- **Save outputs**: Download generated audio before session expires

### Colab Limitations:

- Sessions timeout after ~12 hours of inactivity
- GPU usage is limited (use wisely)
- Files are temporary (download important outputs)

---

**Enjoy using Chatterbox TTS! 🎉**